### Inicialização do ambiante

Para testar o script de geração do mapa de calor, vamos gerar pontos de calor randomicos utlizando as coordenadas da cidade de Blumenau.

Latitude e longitude vem de http://bboxfinder.com/#-27.007138,-49.159698,-26.674459,-48.965378

Acurária do 3º decimal é de ~100m
https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude

In [80]:
# Carrega bibliotecas utilizadas
import folium
import pandas as pd
from folium import plugins
import geopy

In [81]:
ARQUIVO_DADOS = "/home/jvanz/vulneraveis/dados.csv"

#deseja carregar as coordenadas?
carrega_coordenada = False

localizador = geopy.Nominatim(user_agent="heatmapGeocoder")

def get_coordenadas(linha):
    """Função que busca as coordenadas utilizando o localizador definido."""
    localizacao = localizador.geocode(linha["endereco_completo"])
    if localizacao:
        linha["latitude"] = localizacao.latitude
        linha["longitude"] = localizacao.longitude
    return linha

# Precisa buscar as coordenadas novamente?
if carrega_coordenada:
    dados = pd.read_csv(ARQUIVO_DADOS, sep=";")
    #dados = dados.loc[list(range(10))]
    dados["latitude"] = 0
    dados["longitude"] = 0
    # monta um endereco completo para ser utlizado pelo buscador de coordenadas
    dados["endereco_completo"] = dados["logradouro"] + "," + dados["numero"] + "," + dados["bairro"] + ", " + dados["cep"] + ", Blumenau, Santa Catarina, Brasil"

    # procura latitude e longitude dos endereços
    dados = dados.apply(get_coordenadas, axis=1)
    # grava o arquivo com as coordenadas
    dados.to_csv(ARQUIVO_DADOS, index=False)

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

### Geração do mapa de calor

In [ ]:
# Carrega coordenadas dos pontos de calor do arquivo
coordenadas = pd.read_csv(ARQUIVO_DADOS)
coordenadas = coordenadas[["latitude", "longitude"]]
# ignore coordenadas invalidas
coordenadas = coordenadas[(coordenadas.latitude != 0) & (coordenadas.longitude != 0)]

long_min =  coordenadas.longitude.min()
long_max =  coordenadas.longitude.max()
lat_min = coordenadas.latitude.min()
lat_max = coordenadas.latitude.max()

# Plota o gráfico do mapa de calor
m = folium.Map(
    [np.mean([lat_min, lat_max]), np.mean([long_min, long_max])],
    zoom_start=11
)

m.add_child(
    plugins.HeatMap(coordenadas, radius=15)
)

m